# Regiment

### Introduction:

Special thanks to: http://chrisalbon.com/ for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min, avg,max

### Step 2. Create the DataFrame with the following values:

In [3]:
raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'], 
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'], 
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'], 
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}

In [27]:
spark = SparkSession.builder\
                    .appName('regiment')\
                    .getOrCreate()

25/06/06 17:49:48 WARN Utils: Your hostname, kevin-llanos-Type1ProductConfigId resolves to a loopback address: 127.0.1.1; using 192.168.1.92 instead (on interface wlo1)
25/06/06 17:49:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/06 17:49:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/06 17:49:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


25/06/06 17:50:03 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Step 3. Assign it to a variable called regiment.
#### Don't forget to name each column

In [26]:
df = pd.DataFrame(raw_data)

In [28]:
regiment = spark.createDataFrame(df)


In [31]:
regiment.show()

+----------+-------+--------+------------+-------------+
|  regiment|company|    name|preTestScore|postTestScore|
+----------+-------+--------+------------+-------------+
|Nighthawks|    1st|  Miller|           4|           25|
|Nighthawks|    1st|Jacobson|          24|           94|
|Nighthawks|    2nd|     Ali|          31|           57|
|Nighthawks|    2nd|  Milner|           2|           62|
|  Dragoons|    1st|   Cooze|           3|           70|
|  Dragoons|    1st|   Jacon|           4|           25|
|  Dragoons|    2nd|  Ryaner|          24|           94|
|  Dragoons|    2nd|    Sone|          31|           57|
|    Scouts|    1st|   Sloan|           2|           62|
|    Scouts|    1st|   Piger|           3|           70|
|    Scouts|    2nd|   Riani|           2|           62|
|    Scouts|    2nd|     Ali|           3|           70|
+----------+-------+--------+------------+-------------+



### Step 4. What is the mean preTestScore from the regiment Nighthawks?  

In [34]:
regiment.filter(col('regiment')=='Nighthawks').agg(avg(col('postTestScore')).alias('Promedio')).show()

+--------+
|Promedio|
+--------+
|    59.5|
+--------+



In [38]:
df_night = df.loc[df['regiment']=='Nighthawks', :]

promedio_night = df_night['postTestScore'].mean()

print('El promedio de prestescore para nightwaks %s' %promedio_night)

El promedio de prestescore para nightwaks 59.5


### Step 5. Present general statistics by company

In [41]:
from pyspark.sql.functions import avg, stddev, min, max, count

grouped_company = regiment.groupBy(col('company'))

estadisticas = grouped_company.agg(
    count("*").alias("n"),
    avg("preTestScore").alias("_prom"),
    stddev("preTestScore").alias("_std"),
    min("preTestScore").alias("_min"),
    max("preTestScore").alias("_max"),
    avg("postTestScore").alias("_prom"),
    stddev("postTestScore").alias("_std"),
     min("postTestScore").alias("_min"),
    max("postTestScore").alias("_max"),
    avg("postTestScore").alias("_prom"),
)

estadisticas.show()

+-------+---+-----------------+------------------+----+----+------------------+------------------+----+----+------------------+
|company|  n|            _prom|              _std|_min|_max|             _prom|              _std|_min|_max|             _prom|
+-------+---+-----------------+------------------+----+----+------------------+------------------+----+----+------------------+
|    1st|  6|6.666666666666667| 8.524474568362947|   2|  24|57.666666666666664| 27.48575388572536|  25|  94|57.666666666666664|
|    2nd|  6|             15.5|14.652644812456213|   2|  31|              67.0|14.057026712644463|  57|  94|              67.0|
+-------+---+-----------------+------------------+----+----+------------------+------------------+----+----+------------------+



In [42]:
df.groupby('company').describe()

preTestScore                                                      \
               count       mean        std  min   25%   50%    75%   max   
company                                                                    
1st              6.0   6.666667   8.524475  2.0  3.00   3.5   4.00  24.0   
2nd              6.0  15.500000  14.652645  2.0  2.25  13.5  29.25  31.0   

        postTestScore                                                       
                count       mean        std   min    25%   50%   75%   max  
company                                                                     
1st               6.0  57.666667  27.485754  25.0  34.25  66.0  70.0  94.0  
2nd               6.0  67.000000  14.057027  57.0  58.25  62.0  68.0  94.0

### Step 6. What is the mean of each company's preTestScore?

### Step 7. Present the mean preTestScores grouped by regiment and company

### Step 8. Present the mean preTestScores grouped by regiment and company without heirarchical indexing

### Step 9. Group the entire dataframe by regiment and company

### Step 10. What is the number of observations in each regiment and company

### Step 11. Iterate over a group and print the name and the whole data from the regiment